## Task 1

In [1]:
from pyspark import SparkContext

sc = SparkContext()
file = sc.textFile("word_list-3.txt")

In [2]:
# Convert to upper
myfile = file.map(lambda line: line.upper())
# Take a look at the first two lines
myfile.take(2)
'''
My understanding:
The map function convert each line to uppercase
The take function shows the defined number of lines
'''

['THE PROJECT GUTENBERG ETEXT OF MOBY WORD II BY GRADY WARD',
 'COPYRIGHT LAWS ARE CHANGING ALL OVER THE WORLD, BE SURE TO CHECK']

In [3]:
# Count the number of lines
file.count()
'''
My understanding:
The count function counts how many lines
'''

260

In [4]:
# Count the number of word "PROJECT"
myfile.flatMap(lambda line: line.split(" ")).filter(lambda word: word.count("PROJECT")).count()
'''
My understanding:
lambda function returns a list of all words separated by a white space
filter function only returns the number of word "PROJECT" of each element of the list
the count function sums up the whole list
'''

32

In [5]:
# Count the number of words in the dataset
myfile.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda a,b: a+b).collect()
'''
My understanding:
lambda function returns a list of all words separated by a white space
map function return a tuple of a word and 1 (initial count)
reducebykey function adds the same word by the word count from different storage from memory
collect function combines all of them
'''

[('GUTENBERG', 13),
 ('ETEXT', 21),
 ('OF', 72),
 ('MOBY', 5),
 ('WORD', 6),
 ('II', 5),
 ('GRADY', 2),
 ('COPYRIGHT', 8),
 ('LAWS', 2),
 ('ARE', 15),
 ('OVER', 4),
 ('BE', 15),
 ('TO', 55),
 ('CHECK', 1),
 ('FOR', 21),
 ('YOUR', 12),
 ('COUNTRY', 1),
 ('FILES!!!', 1),
 ('PLEASE', 5),
 ('THIS', 42),
 ('HEADER.', 1),
 ('WE', 14),
 ('YOU', 45),
 ('KEEPING', 1),
 ('ELECTRONIC', 1),
 ('OPEN', 1),
 ('READERS.', 2),
 ('', 270),
 ('SHOULD', 4),
 ('WHEN', 4),
 ('OPENS', 1),
 ('CHANGE', 1),
 ('PERMISSION.', 1),
 ('CAREFULLY', 1),
 ('CHOSEN', 1),
 ('USERS', 1),
 ('THEY', 5),
 ('NEED', 4),
 ('ABOUT', 6),
 ('WHAT', 2),
 ('CAN', 5),
 ('LEGALLY', 2),
 ('TEXTS.', 1),
 ('GET', 6),
 ('ETEXTS,', 4),
 ('IS', 18),
 ('DONATIONS.', 1),
 ('PRESENTLY,', 2),
 ('CONTRIBUTIONS', 3),
 ('BEING', 2),
 ('SOLICITED', 2),
 ('IN:', 2),
 ('IDAHO,', 2),
 ('WYOMING,', 2),
 ('COLORADO,', 2),
 ('SOUTH', 2),
 ('IOWA,', 2),
 ('INDIANA,', 2),
 ('AS', 15),
 ('REQUIREMENTS', 2),
 ('OTHER', 16),
 ('ADDITIONS', 2),
 ('LIST', 9),
 

## 2. Create spark RDD from external dataset(shakespeare.txt)

####     Change all words to lowercase and show the first 5 lines.


In [6]:
# Read the file
file = sc.textFile("shakespeare-3.txt")

In [7]:
# Convert to lower
myfile = file.map(lambda line: line.lower())
# Show the first 5 lines
myfile.take(5)
'''
My understanding:
The map function convert each line to lowercase
The take function shows the defined number of lines
'''

['the project gutenberg ebook of the complete works of william shakespeare, by ',
 'william shakespeare',
 '',
 'this ebook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  you may copy it, give it away or']

In [8]:
# Count the total number of words
myfile.flatMap(lambda line: line.split(" ")).count()
'''
My understanding:
The lambda function splits each line to words separated by white spaces
The count function counts all elements (words) in the list
'''

1410759

In [9]:
# Count the number of word "is"
myfile.flatMap(lambda line: line.split(" ")).filter(lambda word: word.count("is")).count()
'''
My understanding:
lambda function returns a list of all words separated by a white space
filter function only returns the number of word "is" of each element of the list
the count function sums up the whole list
'''

36799

In [10]:
# Count the number of unique words in the dataset.
myfile.flatMap(lambda line: line.split(" ")).distinct().count()
'''
My understanding:
The lambda function splits each line to words separated by white spaces
The distinct function removes duplicated words
The count function counts all elements (words) in the list
'''

59739

## 3. Create Spark dataframe from hotel_booking data and execute some query.

In [11]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.master("local")
    .appName("Colab")
    .config("spark.ui.port", "4050")
    .getOrCreate()
)
# Create a Spark SQL object
df = spark.read.format("com.dtabricks.spark.csv").option('mode', 'DROPMALFORMED').option('header', True).option('inferschema', True).csv('hotel_bookings-1.csv')

In [12]:
# Show stats of column "adults"
df.describe('adults').show()
'''
My understanding:
The describe function's parameter is the name of the column
'''

+-------+------------------+
|summary|            adults|
+-------+------------------+
|  count|            119390|
|   mean|1.8564033838679956|
| stddev|0.5792609988327535|
|    min|                 0|
|    max|                55|
+-------+------------------+



In [13]:
# Count total number of cancels by hotel
df.groupBy('hotel').sum('is_canceled').show()
'''
My understanding:
The groupBy function's parameter is the name of the column that will be grouped by
The sum function's parameter sums all rows that the value of column "is_canceled" is True
'''

+------------+----------------+
|       hotel|sum(is_canceled)|
+------------+----------------+
|Resort Hotel|           11122|
|  City Hotel|           33102|
+------------+----------------+



In [14]:
# Register the DataFrame as a global temporary view.
df.createGlobalTempView('hotelbook')
'''
My understanding:
df can be queried using SQL syntax
'''

In [15]:
# Use query to count number of records is reservation_status=”canceled”.
spark.sql('select count(reservation_status) from global_temp.hotelbook where reservation_status == \'Canceled\'').show()
'''
My understanding:
Select the number of reservation_status from table hotelbook that the string value of reservation_status is "Canceled"
'''

+-------------------------+
|count(reservation_status)|
+-------------------------+
|                    43017|
+-------------------------+



In [16]:
# Use query to count the number of agent group by hotel.
spark.sql('select sum(agent) from global_temp.hotelbook group by hotel').show()
'''
My understanding:
Select the total of agent from table hotelbook grouped by each hotel
'''

+----------+
|sum(agent)|
+----------+
| 6929877.0|
| 2003876.0|
+----------+



In [17]:
# Use query to count the number of babies when babies are greater than 0 by year.
spark.sql('select arrival_date_year, count(babies) from global_temp.hotelbook where babies > 0 group by arrival_date_year').show()
'''
My understanding:
Select arrival_date_year and the number of babies from table hotelbook that the number of babies is greater than zero grouped by arrival_date_year
'''

+-----------------+-------------+
|arrival_date_year|count(babies)|
+-----------------+-------------+
|             2015|          213|
|             2016|          446|
|             2017|          258|
+-----------------+-------------+



In [19]:
# Use query to sort the number of canceled by country in decreasing order.
spark.sql('select country, sum(is_canceled) as all_canceled from global_temp.hotelbook group by 1 order by 2 desc').show()
'''
My understanding:
Select country abbreviation / code and the total number of canceled bookings with an alias all_canceled from table hotelbook grouped by country and ordered by all_canceled
'''

+-------+------------+
|country|all_canceled|
+-------+------------+
|    PRT|       27519|
|    GBR|        2453|
|    ESP|        2177|
|    FRA|        1934|
|    ITA|        1333|
|    DEU|        1218|
|    IRL|         832|
|    BRA|         830|
|    USA|         501|
|    BEL|         474|
|    CHN|         462|
|    CHE|         428|
|    NLD|         387|
|     CN|         254|
|    RUS|         239|
|    AUT|         230|
|    SWE|         227|
|    POL|         215|
|    AGO|         205|
|    NOR|         181|
+-------+------------+
only showing top 20 rows

